In [2]:
import tensorflow as tf
import keras
from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Add, MaxPooling1D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam
from keras.engine.network import Network
from keras import backend as K

import keras.backend.tensorflow_backend as KTF

from keras.initializers import glorot_normal,orthogonal

from keras.callbacks import EarlyStopping,TensorBoard
import csv

from PIL import Image

%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import cv2
import os
import random

from tqdm import tqdm

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# sess = tf.Session(config=config)
# K.set_session(sess)


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [3]:
#どんくらいで学習させるか関連
timesteps=10 #一回のLSTMに入れる値の数
camera=5 #カメラの数

rate = 10 #飛ばすフレーム数（30FPSを30/rateのFPSの動画に疑似変換する．）

#画像関連 
channels=3
img_width=50
img_height=50

#LSTMなどで用いる値
epochs = 100 # エポック数 
batch_size = 5 # バッチサイズ

In [4]:
class DataLoader:
    def __init__(self, camera_num, file_name, timesteps, width, height):
        self.camera_num = camera_num
        self.file_name =file_name
        self.samples=0
        self.timesteps=timesteps
        self.width=width
        self.height=height
        self.save_numpy()
        
    def save_numpy(self):
        self.samples = int(self.get_sample()/rate)
        if not (os.path.isfile(self.file_name+"train/label_"+str(rate)+".npy")):
            self.make_numpy_learnData()
            self.make_numpy_labelData()
        self.learnData=np.load(self.file_name+"train/learn_"+str(rate)+".npy")
        self.labelData=np.load(self.file_name+"train/label_"+str(rate)+".npy")
    
    def get_sample(self):
        video_path = self.file_name+"video/0.mp4"
        cap = cv2.VideoCapture(video_path)
        num = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        return num
    
    def make_numpy_labelData(self):
        labelData=[[],[],[],[],[]]
        test=[0,0,0,0,0]
        csv_selection=csv.reader(open(self.file_name+"video/mintime_optimize.csv", 'r'))
        for i,row2 in enumerate(csv_selection):
            if(i%rate==0 and i > (self.timesteps-1)*rate-1):
                for j in range(self.camera_num):
                    if j == int(row2[1]):
                        labelData[j].append([1])
                    else:
                        labelData[j].append([0])
                test[int(row2[1])]+=1
        
        
        tmp_list = np.array(labelData)
        tmp_list = tmp_list.astype(np.float)
        np.save(self.file_name+"train/label_"+str(rate)+".npy", tmp_list)
        print(tmp_list.shape)
        print(test)
    
    
    def make_numpy_learnData(self):
        learnData=[]
        for i in range(0, self.camera_num):
            print()
            img_list=[]
            video_path = self.file_name+"video/"+str(i)+".mp4"
            cap = cv2.VideoCapture(video_path)
            video_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            for j in range(0,video_len):
                
                pro_size=20
                bar = int(j*pro_size/video_len)
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(j / video_len * 100.)
                print('\r{0}/{1} [{2}] {3}%'.format((i+1), self.camera_num, pro_bar, percent), end='')
                
                ret, frame = cap.read()
                if(j%rate==0):
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    img=Image.fromarray(frame)
                    img = img.resize((self.width, self.height))
                    x = np.array(img, dtype=np.float32)
                    x = x / 255.
                    img_list.append(x)
            learnData.append(img_list)
            
        tmp_list=np.array(learnData)
        tmp_list=tmp_list.astype(np.float)
        np.save(self.file_name+"train/learn_"+str(rate)+".npy", tmp_list)
        print(tmp_list.shape)


    def make_learndata(self):
        data_name=self.file_name+"train/learn_0_"+str(rate)+".npy"
        if (os.path.isfile(data_name)):
            test=np.load(data_name)
        else:
            all_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
            tmp_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
            for i in range(0, self.labelData.shape[1]):
                #まずはすべての分だけ回す
                pro_size=20
                bar = int(i*pro_size/self.labelData.shape[1])
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(i / self.labelData.shape[1] * 100.)
                print('\r [{0}] {1}%'.format(pro_bar, percent), end='')

                for camera in range(0, self.camera_num):
                    #カメラの台数だけ回す
                    #tmp_list[camera]の同じ引数（番号）には同じ時間軸のsequenceが入っている．
                    first_list =np.empty((0, self.width, self.height, 3), np.float)
                    for l in range(0, self.timesteps):
                        #timesteps分のndarrayを作る．
                        img=self.learnData[camera,i+l]
                        first_list =np.append(first_list, [img], axis=0)
                    #あるカメラについて，１つのsequenceがfirst_listに出来上がっている状態．
                    tmp_list[camera]=np.append(tmp_list[camera], [first_list], axis=0)


                if(tmp_list[0].shape[0]>100 or i == self.labelData.shape[1]-1):
                    #処理時間短縮のため，sequenceがtmp_listに溜まってきたら，全体に統合してまたtmp_listを初期化する．
                    for camera in range(0, self.camera_num):
                        all_list[camera]=np.append(all_list[camera], tmp_list[camera], axis=0)
                        tmp_list[camera] = np.empty((0,self.timesteps, self.width, self.height, 3), np.float)
            
            for camera in range(0, self.camera_num):
                print(all_list[camera].shape)
                data_name=self.file_name+"train/learn_"+str(camera)+"_"+str(rate)
                np.save(data_name, all_list[camera])
    
    
    def make_labeldata(self):
        data_name=self.file_name+"train/final_label_"+str(rate)+"_.npy"
        if (os.path.isfile(data_name)):
            test=np.load(data_name)
        else:
            print(self.labelData.shape)
            np.save(data_name, self.labelData)
    
    
    def get_learndata(self, num):
        data_name=self.file_name+"train/learn_"+str(num)+"_"+str(rate)+".npy"
        test=np.load(data_name)
        return test
    
    
    def get_labeldata(self):
        data_name=self.file_name+"train/final_label_"+str(rate)+"_.npy"
        test=np.load(data_name)
        return test
    
    
    def show(self):
        print(0,self.get_learndata(0).shape)
        print(1,self.get_learndata(1).shape)
        print(2,self.get_learndata(2).shape)
        print(3,self.get_learndata(3).shape)
        print(4,self.get_learndata(4).shape)
        print(self.get_labeldata().shape)

In [5]:
data = DataLoader(camera, "./data/20190427/", timesteps, img_width, img_height)


5/5 [=================== ] 99.993827%(5, 1620, 50, 50, 3)
(5, 1611, 1)
[268, 61, 95, 221, 966]


In [6]:
data.make_learndata()
data.make_labeldata()
data.show()

 [=================== ] 99.937927%(1611, 10, 50, 50, 3)
(1611, 10, 50, 50, 3)
(1611, 10, 50, 50, 3)
(1611, 10, 50, 50, 3)
(1611, 10, 50, 50, 3)
(5, 1611, 1)
0 (1611, 10, 50, 50, 3)
1 (1611, 10, 50, 50, 3)
2 (1611, 10, 50, 50, 3)
3 (1611, 10, 50, 50, 3)
4 (1611, 10, 50, 50, 3)
(5, 1611, 1)


In [7]:
class Resnet18 :
    ##
    ##　Resnet-18の作成
    ##
    def __init__(self, width, height, channels):
        self.img_rows=height
        self.img_cols=width
        self.img_channels=channels
    
    def rescell(self, data, filters, kernel_size, option=False):
        strides=(1,1)
        if option:
            strides=(2,2)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=strides,padding="same")(data)
        x=BatchNormalization()(x)
        x=Activation('relu')(x)
        
        data=Convolution2D(filters=int(x.shape[3]), kernel_size=(1,1), strides=strides, padding="same")(data)
        x=Convolution2D(filters=filters,kernel_size=kernel_size,strides=(1,1),padding="same")(x)
        x=BatchNormalization()(x)
        x=Add()([x,data])
        x=Activation('relu')(x)
        
        return x
    
    def create_resnet18(self):
        inputs=Input(shape=(self.img_rows, self.img_cols, self.img_channels))
        x=Convolution2D(64,(7,7), padding="same", input_shape=(self.img_rows, self.img_cols),activation="relu")(inputs)
        x=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        x=self.rescell(x,64,(3,3))
        
        x=self.rescell(x,128,(3,3),True)
        x=self.rescell(x,128,(3,3))

        x=self.rescell(x,256,(3,3),True)
        x=self.rescell(x,256,(3,3))
        
        x=self.rescell(x,512,(3,3),True)
        x=self.rescell(x,512,(3,3))
        
        model=Model(inputs=inputs,outputs=[x])
        return model

    def get_resnet18(self):
        return self.create_resnet18()

In [8]:
class Prediction :
    def __init__(self, timesteps, n_out, width, height):
        self.maxlen = timesteps
        self.n_out = n_out
        self.width=width
        self.height=height
        self.resnet = Resnet18(width, height, 3).get_resnet18()
        self.sharedLSTMmodel = self.create_sharedLSTMmodel()
        self.sharedRateModel = self.create_sharedRatemodel()
    
    
    def create_sharedLSTMmodel(self):
        inputs = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        x = TimeDistributed(self.resnet,name="resnet")(inputs)
        x = TimeDistributed(GlobalAveragePooling2D(), name="GAP")(x)
        x = TimeDistributed(Dense(512,activation='relu'), name="dense")(x)
        predictions = LSTM(256, batch_input_shape = (None, self.maxlen, 512),
             kernel_initializer = glorot_normal(seed=20181020),
             recurrent_initializer = orthogonal(gain=1.0, seed=20181020), 
             dropout = 0.01, 
             recurrent_dropout = 0.01)(x)
        
        predictions =Reshape((1,256))(predictions)
        shared_layers = Model(inputs, predictions, name="shared_LSTMlayers")
        return shared_layers
    
    
    def create_sharedRatemodel(self):
        inputs = Input(shape=(512,))
        mid_dense = Dense(256, activation='relu')(inputs)
        predictions = Dense(1, activation='sigmoid')(mid_dense)
        shared_layers = Model(inputs, predictions, name="shared_Ratelayers")
        return shared_layers


    def create_model(self):
        model_input1 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input2 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input3 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input4 = Input(shape=(self.maxlen, self.height, self.width, 3))
        model_input5 = Input(shape=(self.maxlen, self.height, self.width, 3))
        
        mid_feature1 = self.sharedLSTMmodel(model_input1)
        mid_feature2 = self.sharedLSTMmodel(model_input2)
        mid_feature3 = self.sharedLSTMmodel(model_input3)
        mid_feature4 = self.sharedLSTMmodel(model_input4)
        mid_feature5 = self.sharedLSTMmodel(model_input5)
        
        
        pooling_vector = keras.layers.concatenate([mid_feature1, mid_feature2, mid_feature3, mid_feature4, mid_feature5], axis=1)
        pooling_vector = MaxPooling1D(pool_size=camera)(pooling_vector)
        
        
        merge_feature1 = keras.layers.concatenate([mid_feature1, pooling_vector], axis=-1)
        merge_feature2 = keras.layers.concatenate([mid_feature2, pooling_vector], axis=-1)
        merge_feature3 = keras.layers.concatenate([mid_feature3, pooling_vector], axis=-1)
        merge_feature4 = keras.layers.concatenate([mid_feature4, pooling_vector], axis=-1)
        merge_feature5 = keras.layers.concatenate([mid_feature5, pooling_vector], axis=-1)
        
        merge_feature1 = Reshape((512,))(merge_feature1)
        merge_feature2 = Reshape((512,))(merge_feature2)
        merge_feature3 = Reshape((512,))(merge_feature3)
        merge_feature4 = Reshape((512,))(merge_feature4)
        merge_feature5 = Reshape((512,))(merge_feature5)
        
        predictions1 = self.sharedRateModel(merge_feature1)
        predictions2 = self.sharedRateModel(merge_feature2)
        predictions3 = self.sharedRateModel(merge_feature3)
        predictions4 = self.sharedRateModel(merge_feature4)
        predictions5 = self.sharedRateModel(merge_feature5)
        
        model = Model(inputs=[model_input1, model_input2, model_input3, model_input4, model_input5], 
                      outputs=[predictions1,predictions2,predictions3,predictions4,predictions5])
        model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
        return model
    
    
    def train(self, x_train, t_train, x_vali, t_vali, batch_size, epochs):
        early_stopping = EarlyStopping(patience=20, verbose=1)
        model = self.create_model()
        hist = model.fit(x=[x_train[0], x_train[1], x_train[2], x_train[3], x_train[4]], y=t_train, batch_size = batch_size,
                         epochs = epochs, validation_data=([x_vali[0], x_vali[1], x_vali[2], x_vali[3], x_vali[4]], t_vali),
                         callbacks = [early_stopping])
        return model, hist
    
    
    def train_keras(self, x_train, t_train, batch_size, epochs):
        early_stopping = EarlyStopping(patience=20, verbose=1)
        model = self.create_model()
        hist = model.fit([x_train[0], x_train[1], x_train[2], x_train[3], x_train[4]], 
                         [t_train[0], t_train[1], t_train[2], t_train[3], t_train[4]],
                         batch_size = batch_size, epochs = epochs, verbose = 1,
                         callbacks = [early_stopping], validation_split = 0.2)
        return model, hist

In [9]:
# モデル定義
pred = Prediction(timesteps, camera, img_width, img_height)
model = pred.create_model()
print(model.summary())












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 10, 50, 50, 3 0                                            
_____________________________________________________________________________________

In [10]:
from tqdm import tqdm
test_learn=[np.empty((data.get_labeldata().shape[0],10, 50, 50, 3), np.float)]*camera
vali_learn=[np.empty((0,10, 50, 50, 3), np.float)]*camera
test_label=data.get_labeldata()
vali_label=np.empty((0,camera), np.float)

print(test_label.shape)

for j in range(0,camera):
    test_learn[j]=data.get_learndata(j)

# for i in tqdm(range(0,int(0.1*test_label.shape[0]))):
#     random_angle = int(random.randint(0,test_label.shape[0]-1))
#     for j in range(0,camera):
#         vali_learn[j]=np.append(vali_learn[j], [test_learn[j][random_angle]],axis=0)
#         test_learn[j]=np.delete(test_learn[j], random_angle,0)
#     vali_label=np.append(vali_label, [test_label[random_angle]], axis=0)
#     test_label=np.delete(test_label, random_angle,0)

(5, 1611, 1)


In [11]:
# 学習
#model , hist = pred.train(test_learn, test_label, vali_learn, vali_label, batch_size, epochs)
model , hist = pred.train_keras(test_learn, test_label, batch_size, epochs)
# テスト
score = model.evaluate(test_learn, test_label, batch_size = batch_size, verbose = 1)
print("score:", score)



Train on 1288 samples, validate on 323 samples
Epoch 1/100


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[50,64,25,25] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node shared_LSTMlayers_9/resnet/batch_normalization_3/FusedBatchNormV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[loss_1/shared_Ratelayers_loss/Mean_3/_2597]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[50,64,25,25] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node shared_LSTMlayers_9/resnet/batch_normalization_3/FusedBatchNormV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oo

In [83]:
def freeze_layers(model):
    for i in model.layers:
        i.trainable = False
        if isinstance(i, Model):
            freeze_layers(i)
    return model

model_freezed = freeze_layers(model)
model_freezed.save('model/file0205.tf')

In [1]:
def plot_history_loss(hist):
    # 損失値(Loss)の遷移のプロット
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()

def plot_history_acc(hist):
    # 精度(Accuracy)の遷移のプロット
    plt.plot(hist.history['acc'],label="loss for training")
    plt.plot(hist.history['val_acc'],label="loss for validation")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.ylim([0, 1])
    plt.show()

plot_history_loss(hist)
plot_history_acc(hist)

NameError: name 'hist' is not defined

In [ ]:
# カメラ番号をランダムに並び替えることでデータの偏りをなくそうというデータ作成プログラム


# def make_learndata(self):
#         data_name=self.file_name+"learn_0_"+str(rate)+".npy"
#         if (os.path.isfile(data_name)):
#             test=np.load(data_name)
#         else:
#             all_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
#             tmp_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
#             for i in range(0, self.labelData.shape[0]):
#                 #まずはすべての分だけ回す
#                 pro_size=20
#                 bar = int(i*pro_size/self.labelData.shape[0])
#                 pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
#                 percent ='{:03f}'.format(i / self.labelData.shape[0] * 100.)
#                 print('\r [{0}] {1}%'.format(pro_bar, percent), end='')
                            
#                 for j in range(0,self.camera_num):
#                     for camera in range(0, self.camera_num):
#                         #カメラの台数だけ回す
#                         #tmp_list[camera]の同じ引数（番号）には角度を同じ分だけずらした同じ時間軸のsequenceが入っている．
#                         first_list =np.empty((0, self.width, self.height, 3), np.float)
#                         for l in range(0, self.timesteps):
#                             #timesteps分のndarrayを作る．
#                             img=self.learnData[camera,i+l]
#                             first_list =np.append(first_list, [img], axis=0)
#                         #あるカメラについて，１つのsequenceがfirst_listに出来上がっている状態．
#                         tmp_camera_num=self.camera_return(camera+j)
#                         tmp_list[tmp_camera_num]=np.append(tmp_list[tmp_camera_num], [first_list], axis=0)
    

#                 if(tmp_list[0].shape[0]>100 or i == self.labelData.shape[0]-1):
#                     #処理時間短縮のため，sequenceがtmp_listに溜まってきたら，全体に統合してまたtmp_listを初期化する．
#                     for camera in range(0, self.camera_num):
#                         all_list[camera]=np.append(all_list[camera], tmp_list[camera], axis=0)
#                         tmp_list[camera] = np.empty((0,self.timesteps, self.width, self.height, 3), np.float)
            
#             for camera in range(0, self.camera_num):
#                 print(all_list[camera].shape)
#                 data_name=self.file_name+"learn_"+str(camera)+"_"+str(rate)
#                 np.save(data_name, all_list[camera])
    
    
#     def make_labeldata(self):
#         data_name=self.file_name+"final_label_"+str(rate)+"_.npy"
#         if (os.path.isfile(data_name)):
#             test=np.load(data_name)
#         else:
#             test = np.empty((0,self.camera_num), np.float)
#             for i in range(0,self.labelData.shape[0]):
#                 for j in range(0, self.camera_num):
#                     tmp=self.camera_return(np.argmax(self.labelData[i])+j)
#                     a_one_hot = np.identity(self.camera_num)[tmp]
#                     print(a_one_hot)
#                     test = np.append(test, [a_one_hot], axis=0)
#             print(test.shape)
#             np.save(data_name, test)


    
#     def camera_return(self, num=0):
#         if(num<self.camera_num):
#             return num
#         else:
#             return num-self.camera_num